In [1]:
#from IPython.display import HTML
#import io
import numpy as np
import pandas as pd

pd.set_option('max_rows', None)

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
sheets = gc.open('RYM 1001 Performances')
round1sheet = sheets.worksheet('Round1')

In [2]:
data = round1sheet.get_all_values()
data = np.array(data)
df = pd.DataFrame(data[1:], columns=data[0], index=range(1,51))
#df.tail(60).style.set_properties(**{'text-align': 'left'})

In [3]:
df = pd.DataFrame(data[1:], columns=data[0], index=range(1,51))

# Make Group column
df = pd.melt(df)
df.columns = ['Group', 'Data']
df['Group'] = df['Group'].str[6:]

# Make additional columns
df["GroupRank"] = df["Data"].str.split(" ", n = 1, expand = True)[0].str[:-1]
df["Name"] = df["Data"].str.split(" in ", n = 1, expand = True)[0].str.split(".", n = 1, expand = True)[1].str.lstrip().str.rstrip()
df['Film'] = df["Data"].str.split(" in ", n = 1, expand = True)[1].str.split("(", n = 1, expand = True)[0].str.lstrip().str.rstrip()
df['VoteDist'] = df["Data"].str.split(")", n = 1, expand = True)[1].str.split("[", n = 1, expand = True)[0].str.strip()
df['Votes'] = df['VoteDist'].str.len()

df['Name'] = df['Name'].where(df['Name'].str[-1:] != ']', df['Name'].str[:-1].str.split('[').str[1])
df['Film'] = df['Film'].where(df['Film'].str[-1:] != ']', df['Film'].str[:-1].str.split('[').str[1])

In [4]:
df.iloc[1000:,0] = 20
df.drop_duplicates(inplace=True)
df.tail()

,Group,Data,GroupRank,Name,Film,VoteDist,Votes
996,20,47. Simone Signoret in Room at the Top (1959) ...,47,Simone Signoret,Room at the Top,413,3
997,20,48. Armin Mueller-Stahl in Shine (1996) 142 [7/3],48,Armin Mueller-Stahl,Shine,142,3
998,20,49. Robert Pattinson in Cosmopolis (2012) 34 [...,49,Robert Pattinson,Cosmopolis,34,2
999,20,"50. Anders Danielsen Lie in Oslo, 31. august [...",50,Anders Danielsen Lie,"Oslo, August 31st",31,2
1000,20,51. Riccardo Cucciolla in Un flic [Dirty Money...,51,Riccardo Cucciolla,Dirty Money,0,1


In [5]:
def digitsum(stringnumber):
  if stringnumber == None:
    return 0
  return sum(int(digit) for digit in stringnumber)

df['Points'] = df['VoteDist'].apply(digitsum) + 0.01*df['Votes'] + 0.001*df['VoteDist'].str.count('4')

df['Rank'] = df['Points'].rank(ascending=False,method='min').astype(int)

df['Points'] = df['Points'].round().astype(int)

df['Votes'] = df['Votes'].where(df['Points'] >= 1, 0)

# Change column order and sort
cols=['Rank','Points','Votes','Name','Film','Group','GroupRank','VoteDist']
df = df[cols].sort_values('Rank',ascending=True)

df.index=range(1,len(df['Rank'])+1)

In [6]:
# Object of type 'int64' is not JSON serializable
df = df.astype('object')
df.head(10)
#df['VoteDist']=df['VoteDist'].astype(int)

,Rank,Points,Votes,Name,Film,Group,GroupRank,VoteDist
1,1,60,17,Daniel Day-Lewis,There Will Be Blood,13,1,43234444343424444
2,2,55,15,Marlon Brando,The Godfather,16,1,334244434444444
3,3,55,15,Toshiro Mifune,Seven Samurai,4,1,434444434443343
4,4,55,14,Al Pacino,Dog Day Afternoon,17,1,44444434444444
5,5,54,16,Al Pacino,The Godfather Part II,3,1,3442342443432444
6,6,54,15,Jack Nicholson,One Flew Over the Cuckoo's Nest,7,1,444443443144443
7,6,54,15,Peter O’Toole,Lawrence of Arabia,14,1,443344442442444
8,8,54,15,Philip Seymour Hoffman,The Master,3,2,442334444434344
9,9,53,16,Henry Fonda,12 Angry Men,3,3,4344244243422434
10,10,53,15,Robert De Niro,Raging Bull,13,3,234444434344442


In [7]:
nominationsheet = sheets.worksheet('Nominations')
nominations = nominationsheet.get_all_values()
nominations = np.array(nominations)
ndf = pd.DataFrame(nominations[1:], columns=nominations[0])

ndf['Name'] = ndf['Name'].str.strip()
ndf['Film'] = ndf['Film'].str.strip()
ndf['Name'] = ndf['Name'].where(ndf['Name'].str[-1:] != ']', ndf['Name'].str[:-1].str.split('[').str[1])
ndf['Film'] = ndf['Film'].where(ndf['Film'].str[-1:] != ']', ndf['Film'].str[:-1].str.split('[').str[1])
ndf['ID'] = ndf['Data'].str.split(n=1,expand=True)[0].astype(int)
ndf.head(10)

,Data,Name,Film,Year,Place of Birth,Gender,Birthday,Deathday,Age,Release Date,Genres,ID
0,166 Charlie Chaplin in The Immigrant (1917),Charlie Chaplin,The Immigrant,1917,UK,2,1889-04-16,1977-12-25,28,1917-06-17,Comedy,166
1,549 Lillian Gish in Broken Blossoms or The Yel...,Lillian Gish,Broken Blossoms or The Yellow Man and the Girl,1919,USA,1,1893-10-14,1993-02-27,25,1919-05-13,"Drama, Romance",549
2,201 Conrad Veidt in The Cabinet of Dr. Caligar...,Conrad Veidt,The Cabinet of Dr. Caligari,1920,Germany,2,1893-01-22,1943-04-03,27,1920-02-26,"Drama, Horror, Thriller, Crime",201
3,"616 Max Schreck in Nosferatu, eine Symphonie d...",Max Schreck,Nosferatu: A Symphony of Horror,1922,Germany,2,1879-09-06,1936-02-20,42,1922-02-17,"Drama, Fantasy, Horror",616
4,344 Harold Lloyd in Safety Last (1923),Harold Lloyd,Safety Last,1923,USA,2,1893-04-20,1971-03-08,29,1923-04-01,"Comedy, Romance, Action",344
5,135 Buster Keaton in Our Hospitality (1923),Buster Keaton,Our Hospitality,1923,USA,2,1895-10-04,1966-02-01,28,1923-11-03,"Comedy, Romance",135
6,"136 Buster Keaton in Sherlock, Jr. (1924)",Buster Keaton,"Sherlock, Jr.",1924,USA,2,1895-10-04,1966-02-01,28,1924-04-17,"Action, Comedy, Mystery",136
7,560 Lon Chaney in He Who Gets Slapped (1924),Lon Chaney,He Who Gets Slapped,1924,USA,2,1883-04-01,1973-07-12,41,1924-11-09,"Drama, Romance, Thriller",560
8,918 ZaSu Pitts in Greed (1924),ZaSu Pitts,Greed,1924,USA,1,1894-01-03,1963-06-07,30,1924-12-04,"Drama, Crime",918
9,259 Emil Jannings in Der Letzte Mann [The Last...,Emil Jannings,The Last Laugh,1924,Switzerland,2,1884-07-23,1950-01-02,40,1924-12-23,Drama,259


In [8]:
df['ID'] = None

for i in range(len(df)):

  try:
    name = df.iloc[i,3].strip()
    film = df.iloc[i,4].strip()
    id = int(ndf[(ndf['Name'] == name) & (ndf['Film'] == film)]['Data'].str.split().iloc[0][0])
    df.iloc[i,8] = id
  except IndexError:
    print(name,film)
    continue
  except ValueError:
    print(name,film,'ValueError')
    continue

df.index = df['ID']
#df = df.drop('ID',axis=1)
df.index.name = None
df.head()

,Rank,Points,Votes,Name,Film,Group,GroupRank,VoteDist,ID
210,1,60,17,Daniel Day-Lewis,There Will Be Blood,13,1,43234444343424444,210
606,2,55,15,Marlon Brando,The Godfather,16,1,334244434444444,606
942,3,55,15,Toshiro Mifune,Seven Samurai,4,1,434444434443343,942
11,4,55,14,Al Pacino,Dog Day Afternoon,17,1,44444434444444,11
16,5,54,16,Al Pacino,The Godfather Part II,3,1,3442342443432444,16


In [9]:
df['Year'] = None
df['Place of Birth'] = None
df['Gender'] = None
df['Birthday'] = None
df['Deathday'] = None
df['Age'] = None
df['Release Date'] = None
df['Genres'] = None

for i, id in enumerate(df.index):
  try:
    df.loc[id,'Year'] = ndf[ndf['ID'] == id]['Year'].iloc[0]
    df.loc[id,'Place of Birth'] = ndf[ndf['ID'] == id]['Place of Birth'].iloc[0]
    df.loc[id,'Gender'] = ndf[ndf['ID'] == id]['Gender'].iloc[0]
    df.loc[id,'Birthday'] = ndf[ndf['ID'] == id]['Birthday'].iloc[0]
    df.loc[id,'Deathday'] = ndf[ndf['ID'] == id]['Deathday'].iloc[0]
    df.loc[id,'Age'] = ndf[ndf['ID'] == id]['Age'].iloc[0]
    df.loc[id,'Release Date'] = ndf[ndf['ID'] == id]['Release Date'].iloc[0]
    df.loc[id,'Genres'] = ndf[ndf['ID'] == id]['Genres'].iloc[0]
  except IndexError:
    print(i)
    continue

df.head()

,Rank,Points,Votes,Name,Film,Group,GroupRank,VoteDist,ID,Year,Place of Birth,Gender,Birthday,Deathday,Age,Release Date,Genres
210,1,60,17,Daniel Day-Lewis,There Will Be Blood,13,1,43234444343424444,210,2007,UK,2,1957-04-29,None,50,2007-09-27,Drama
606,2,55,15,Marlon Brando,The Godfather,16,1,334244434444444,606,1972,USA,2,1924-04-03,2004-07-01,47,1972-03-14,"Drama, Crime"
942,3,55,15,Toshiro Mifune,Seven Samurai,4,1,434444434443343,942,1954,China,2,1920-04-01,1997-12-24,34,1954-04-26,"Action, Drama"
11,4,55,14,Al Pacino,Dog Day Afternoon,17,1,44444434444444,11,1975,USA,2,1940-04-25,None,35,1975-09-19,"Crime, Drama, Thriller"
16,5,54,16,Al Pacino,The Godfather Part II,3,1,3442342443432444,16,1974,USA,2,1940-04-25,None,34,1974-12-20,"Drama, Crime"


In [10]:
cols = ['Rank', 'Points', 'Votes', 'Name', 'Film', 'Group', 'GroupRank', 'VoteDist', 'Year', 'Place of Birth', 'Gender', 'Birthday', 'Deathday', 'Age', 'Release Date', 'Genres', 'ID']
df = df[cols]

round1compiledsheet = gc.open('RYM 1001 Performances').worksheet('Round1Compiled')

def getListIndex(nrow, ncol,row_pos, col_pos):
    list_pos = row_pos*ncol + col_pos
    return(list_pos)

count_row = df.shape[0]
count_col = df.shape[1]

# note this outputs data from the 2nd row
cell_list = round1compiledsheet.range(2,1,count_row+1,count_col)

for row in range(0,count_row):
    for col in range(0,count_col):
        list_index = getListIndex(count_row, count_col, row, col)
        cell_list[list_index].value = df.iloc[row,col]

round1compiledsheet.update_cells(cell_list)

{'spreadsheetId': '1aDqnSxAtr2jMKpfs6httGwOXAe5bkQPkB0aw7wRzI3Y',
 'updatedCells': 17017,
 'updatedColumns': 17,
 'updatedRange': 'Round1Compiled!A2:Q1002',
 'updatedRows': 1001}